In [9]:
import csv
import numpy as np
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import *
from sklearn.svm import SVR
from sklearn import svm
from sklearn.metrics import mean_absolute_error,mean_squared_error
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import warnings; warnings.simplefilter('ignore')
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import explained_variance_score

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import make_blobs
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score

In [11]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

# Pre-processing the Data

In [12]:
def pre_process_data(data,null_threshold):
    data.drop(columns=['Unixtime','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 <= data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(axis=0,inplace=True)
    return data

# Removing columns based on the dependent column

In [13]:
def dependent_column(data,column):
    cols = [col for col in data.columns if "next" not in col.lower()]
    cols.append(column)
    data = data[cols]
    return (data,column)

# ---

In [14]:
kernels = ['RBF', 'Sigmoid','Linear']
#A function which returns the corresponding SVC model

def getClassifier(ktype):
    if ktype == 0:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 1:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 2:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

In [15]:
target = []
def appl(dataframe,x):
    for i in range(0,dataframe.shape[0]):
        if(dataframe.iloc[i].loc['Close Price GR'] > x):
            target.append(1)
        else:
            target.append(0)
    dataframe['Target'] = target
#     print(dataframe)
    X = dataframe.iloc[:,0:97].values
    Y = dataframe.iloc[:,98].values
#     print(Y)
    bestprecision = 0.0
    for i in range(2):
        # Separate data into test and training sets
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
        # Train a SVC model using different kernal
        svclassifier = getClassifier(i) 
        svclassifier.fit(X_train, y_train)
        # Make prediction
        y_pred = svclassifier.predict(X_test)
        # Evaluate our model
        print("Evaluation:", kernels[i], "kernel")
        print(classification_report(y_test,y_pred))
        precision = 0.0
        precision = precision_score(y_test,y_pred)
        if(precision > bestprecision):
            bestprecision = precision
            kernel = kernels[i]
#     print(bestprecision)
#     print(kernel)
    from sklearn.model_selection import GridSearchCV 
    param_grid = {'C': [0.1, 1, 10, 100, 1000],  
                  'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
                  'kernel': ['rbf']}  
    
#     X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 

    # fitting the model for grid search 
    grid.fit(X_train, y_train)
    grid_predictions = grid.predict(X_test) 

    # print classification report 
    print(classification_report(y_test, grid_predictions))

In [16]:
%%time
for filename in os.listdir(os.path.join(path,"Data/Stock")):
    if filename.startswith("mod") and filename.endswith("GR.csv"):
        print(filename)
        name = filename.split(".")[0]
        df = pd.read_csv(os.path.join(path,"Data\Stock\\" + filename))
        df = pre_process_data(df,60)
        dataframe, y = dependent_column(df, "Next Day Close Price GR")
        
        appl(dataframe,0.01)
        confusion_df = pd.DataFrame(confusion_matrix(y_test,y_pred),
             columns=["Predicted Class " + str(class_name) for class_name in [0,1]],
             index = ["Class " + str(class_name) for class_name in [0,1]])

        print(confusion_df)
        break

mod500112GR.csv
Evaluation: RBF kernel
              precision    recall  f1-score   support

           0       0.70      1.00      0.83       569
           1       0.00      0.00      0.00       241

    accuracy                           0.70       810
   macro avg       0.35      0.50      0.41       810
weighted avg       0.49      0.70      0.58       810

Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

           0       0.70      1.00      0.83       569
           1       0.00      0.00      0.00       241

    accuracy                           0.70       810
   macro avg       0.35      0.50      0.41       810
weighted avg       0.49      0.70      0.58       810

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.709, total=   0.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.709, total=   0.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


...

[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.709, total=   0.7s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.709, total=   0.7s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.706, total=   0.8s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.706, total=   0.8s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.708, total=   0.7s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.709, total=   0.6s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.709, total=   0.7s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .

[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.706, total=   0.6s
[CV] C=1000, gamma=0.001, kernel=rbf .................................
[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.708, total=   0.6s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.709, total=   0.6s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.709, total=   0.6s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.706, total=   0.6s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.706, total=   0.6s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.708, total=   0.6s


[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:  1.4min finished


              precision    recall  f1-score   support

           0       0.70      1.00      0.83       569
           1       0.00      0.00      0.00       241

    accuracy                           0.70       810
   macro avg       0.35      0.50      0.41       810
weighted avg       0.49      0.70      0.58       810



NameError: name 'y_test' is not defined